# Running a job

Below are example submission scripts for several different codes.

These examples copy the run data from the submission directory in `/campaign/` area to `$BURSTBUFFER` before the run, and copy any files back to `/campaign/` afterwards. The script then cleans up by removing the `working_directory` in the `$BURSTBUFFER` area (this will be done automatically, but is perhaps a good habit to get into for furture developments in terms of storage were this may not be the case).


Obviously there are an infinite number of ways to structure the logic in your submission scripts in order to handle the data transfer between `/campaign/` and `$BURSTBUFFER`.

The following examples can be adapted to your own needs. They should be submitted from the `/campaign/` storage area, they will create a temporary working directory in `$BURSTBUFFER/$SLURM_JOB_ID` and copy the contents of the submission folder over to the working directory. After the run it copies the contents of the working directory in `$BURSTBUFFER/SLURM_JOB_ID` back to the submission directory in `/campaign/`. It then cleans up the working directory.



## Account code

The account code used in your run script `#SBATCH --account=ACCOUNT_CODE` should match the resource allocation you wish to run your job against. If you don't know your resource allocation code check the research computing account managemnet portal at `rcam.bath.ac.uk` (you need to be on the University's VPN All traffic), or ask your account administrator. 

## QOS

The quality of service (QOS) set in your run script with e.g. `#SBATCH --qos=spot-hbv3-120` should match the partition name you are submitting to. 



## Openfoam

We will run an example job using the OpenFoam 7 module for 

`OpenFoam_hbv3/gcc_9.2.0/openmpi_4.1.0/v7`

Let's load the module:
```bash
module load OpenFoam_hbv3/gcc_9.2.0/openmpi_4.1.0/v7
```

Source the OpenFoam bashrc:

```bash
source $foamDotFile```

Lets switch to our scratch space:

```bash
cd $SCRATCH
```


And copy over the run files:
``` bash
cp -r $WM_PROJECT_DIR/tutorials/multiphase/interFoam/laminar/damBreak ./```

An example run script:
    
```bash
#!/bin/bash
#SBATCH --account=prj3_phase1
#SBATCH --job-name=JOB_NAME
#SBATCH --output=JOB_NAME.%j.o
#SBATCH --error=JOB_NAME.%j.e
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=120
#SBATCH --partition=spot-hbv3-120
#SBATCH --qos=spot-hbv3-120
#SBATCH --time=04:00:00

module purge
module load OpenFoam_hbv3/gcc_9.2.0/openmpi_4.1.0/v7

# As an example of moving input data from /campaign
# to scratch we will create and mirror the submit
# directory structure on scratch:

campaigndir=$(pwd)

echo $campaigndir | if grep -q campaign; then 
    # Copy any inputs required to the work directory:
    # excluding and files with a JOB_NAME prefix, so your
    # output and error files don't get over written on the copy back
    rsync -aP --exclude=JOB_NAME.* $campaigndir/* $BURSTBUFFER
    cd $BURSTBUFFER;
    echo  "Work directory" $BURSTBUFFER ;
    
    # source the foamDotFile and do the run
    . $foamDotFile
    ./Allrun

    # Copy back results.
    srun cp -Rf $BURSTBUFFER/*  $campaigndir/
    #Clean up scratch
    rm -rf $BURSTBUFFER
else
     echo "Not submitting from campaign. Check your submission script."
     exit 1;
fi


```

Lets switch into the folder 
```bash
cd damBreak
```

And after copying the above runscript into `run_job.slm` lets submit the job:

```bash
sbatch run_job.slm
```

### Interactive job

It is also possible to obtain a compute node and work interactively. 

Issuing the following command in the terminal will request a spot-hbv3-120 compute instance for 6 hours:

```
srun --partition spot-hbv3-120 --nodes 1  --account prj4_phase1  --qos spot-hbv3-120 --job-name "interactive" --cpus-per-task 120 --time 6:00:00 --pty bash
```